In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter
import random
import pickle
from sklearn.model_selection import train_test_split
from scipy.stats import ks_2samp
import random
random.seed(123)

In [2]:
%%time
df = pd.read_excel('card transactions.xlsx')

CPU times: user 14.6 s, sys: 230 ms, total: 14.8 s
Wall time: 15 s


In [3]:
# summary
numrecords = len(df)
stat_df = pd.DataFrame(df.isnull().sum(axis = 0))
stat_df = stat_df.reset_index().rename(columns = {'index':'variable', 0 :"# NaN"})
stat_df["% populated"] = (1 - stat_df["# NaN"]/numrecords)*100
stat_df["# unique"] = ''

for i in range(len(list(df))):
    stat_df["# unique"][i] = df[list(df)[i]].nunique()
    
stat_df

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


variable  # NaN  % populated # unique
0             Recnum      0   100.000000    96753
1            Cardnum      0   100.000000     1645
2               Date      0   100.000000      365
3           Merchnum   3375    96.511736    13091
4  Merch description      0   100.000000    13126
5        Merch state   1195    98.764896      227
6          Merch zip   4656    95.187746     4567
7          Transtype      0   100.000000        4
8             Amount      0   100.000000    34909
9              Fraud      0   100.000000        2

In [4]:
# stat_df.to_excel('stat_df.xlsx')

In [5]:
##Identify any exclusions, bad records

# Remove single large transaction outlier in Amount
df = df[df['Amount']<3000000]

# Remove all but P transaction type
df = df[df['Transtype'] == 'P']

len(df)

96397

In [6]:
# --------------- Fill in NAs ----------

### Merchnum 
# Transform 0 values into NA
df.loc[df['Merchnum'] == '0', 'Merchnum'] = np.NaN

# fill most frequenct merchnum using merch description
dict1 = df.set_index('Merch description')['Merchnum'].to_dict()
df['Merchnum'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill most frequenct merchnum in this zip
dict1 = dict(zip(df['Merch zip'].dropna(), df['Merchnum']))
df['Merchnum'].fillna(df['Merch zip'].map(dict1), inplace = True)

pd.isnull(df['Merchnum']).sum()


1414

In [7]:
### Merch state and zip
numrecords - (df['Merch state'].isna() == df['Merch zip'].isna()).sum()


3810

In [8]:
### Merch zip
# df.groupby(['Merch description','Merch zip']).size()

# fill NA in Merch zip using Merch description
dict1 = df.set_index('Merch description')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch zip using Merchnum
dict1 = df.set_index('Merchnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch zip using Cardnum
dict1 = df.set_index('Cardnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Cardnum'].map(dict1), inplace = True)
# It will automatically choose zip with highest frequency

pd.isnull(df['Merch zip']).sum()


543

In [9]:
dic = dict(zip(df['Cardnum'], df['Merch zip']))
dic[5142310525]

60108.0

In [10]:
### Merch state
# df.groupby(['Merch description','Merch state']).size()
# Each Merch description has one matched state

# fill NA in Merch state using Merch description
dict1 = df.set_index('Merch description')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch state using Merchnum
dict1 = df.set_index('Merchnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch state using Merch zip

dict1 = dict(zip(df['Merch zip'].dropna(), df['Merch state']))
df['Merch state'].fillna(df['Merch zip'].map(dict1), inplace = True)

# fill NA in Merch state using Cardnum
dict1 = df.set_index('Cardnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Cardnum'].map(dict1), inplace = True)

pd.isnull(df['Merch state']).sum()


122

In [11]:
# Drop remained NA in three variables
df = df.dropna()
len(df)

94631

In [12]:
# df.groupby(['Cardnum','Date'])['Amount'].value_counts()
# df[df['Cardnum'] == 5142190439]

In [13]:
# --------------- Create variables ----------
### Amount expert variables

# Cardnum
# sum the amount at the same day first and then rolling
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')



In [14]:
# (closed = 'left') in rolling function means when calculating past n days we exlude today's transactions

def amount_mean_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').mean(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

# def amount_max_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).max(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

# def amount_median_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).median(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

def amount_sum_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').sum(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

In [15]:
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')

new_df = amount_mean_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_avg')
# new_df = amount_max_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_max')
# new_df = amount_median_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_median')
new_df = amount_sum_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_sum')
new_df['Amount_cn_1d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act-median'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_median']
new_df['Amount_cn_1d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act/max'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_max']
new_df['Amount_cn_1d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_sum']
# new_df['Amount_cn_1d_act/median'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_median']

new_df = amount_mean_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_sum')
new_df['Amount_cn_4d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_sum')
new_df['Amount_cn_7d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_sum')
new_df['Amount_cn_14d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_14d_avg']
new_df['Amount_cn_14d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_14d_avg']
new_df['Amount_cn_14d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_14d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '30D',rsuffix = '_cn_30d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '30D',rsuffix = '_cn_30d_sum')
new_df['Amount_cn_30d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_30d_avg']
new_df['Amount_cn_30d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_30d_avg']
new_df['Amount_cn_30d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_30d_sum']

In [16]:
df2 = df.groupby(['Merchnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Merchnum','Date'])['Amount'].sum(),on = ['Merchnum','Date'], rsuffix = '_mn_actual')

new_df = amount_mean_func(column = 'Merchnum',window = '1D',rsuffix = '_mn_1d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '1D',rsuffix = '_mn_1d_sum')
new_df['Amount_mn_1d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_1d_avg']
new_df['Amount_mn_1d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_1d_avg']
new_df['Amount_mn_1d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_1d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '4D',rsuffix = '_mn_4d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '4D',rsuffix = '_mn_4d_sum')
new_df['Amount_mn_4d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_4d_avg']
new_df['Amount_mn_4d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_4d_avg']
new_df['Amount_mn_4d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_4d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '7D',rsuffix = '_mn_7d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '7D',rsuffix = '_mn_7d_sum')
new_df['Amount_mn_7d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_7d_avg']
new_df['Amount_mn_7d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_7d_avg']
new_df['Amount_mn_7d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_7d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '14D',rsuffix = '_mn_14d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '14D',rsuffix = '_mn_14d_sum')
new_df['Amount_mn_14d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_14d_avg']
new_df['Amount_mn_14d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_14d_avg']
new_df['Amount_mn_14d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_14d_sum']

new_df = amount_mean_func(column = 'Merchnum',window = '30D',rsuffix = '_mn_30d_avg')
new_df = amount_sum_func(column = 'Merchnum',window = '30D',rsuffix = '_mn_30d_sum')
new_df['Amount_mn_30d_act-avg'] = new_df['Amount_mn_actual'] - new_df['Amount_mn_30d_avg']
new_df['Amount_mn_30d_act/avg'] = new_df['Amount_mn_actual']/new_df['Amount_mn_30d_avg']
new_df['Amount_mn_30d_act/sum'] = new_df['Amount_mn_actual']/new_df['Amount_mn_30d_sum']

In [17]:
df2 = df.groupby(['Cardnum','Merchnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merchnum','Date'])['Amount'].sum(),on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_1d_sum')
new_df['Amount_cn_mn_1d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_1d_avg']
new_df['Amount_cn_mn_1d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_1d_avg']
new_df['Amount_cn_mn_1d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_4d_sum')
new_df['Amount_cn_mn_4d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_4d_avg']
new_df['Amount_cn_mn_4d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_4d_avg']
new_df['Amount_cn_mn_4d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_7d_sum')
new_df['Amount_cn_mn_7d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_7d_avg']
new_df['Amount_cn_mn_7d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_7d_avg']
new_df['Amount_cn_mn_7d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_14d_sum')
new_df['Amount_cn_mn_14d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_14d_avg']
new_df['Amount_cn_mn_14d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_14d_avg']
new_df['Amount_cn_mn_14d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merchnum'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merchnum','Date'], rsuffix = '_cn_mn_30d_sum')
new_df['Amount_cn_mn_30d_act-avg'] = new_df['Amount_cn_mn_actual'] - new_df['Amount_cn_mn_30d_avg']
new_df['Amount_cn_mn_30d_act/avg'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_30d_avg']
new_df['Amount_cn_mn_30d_act/sum'] = new_df['Amount_cn_mn_actual']/new_df['Amount_cn_mn_30d_sum']

In [18]:
df2 = df.groupby(['Cardnum','Merch zip','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merch zip','Date'])['Amount'].sum(),on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_1d_sum')
new_df['Amount_cn_zip_1d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_1d_avg']
new_df['Amount_cn_zip_1d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_1d_avg']
new_df['Amount_cn_zip_1d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_4d_sum')
new_df['Amount_cn_zip_4d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_4d_avg']
new_df['Amount_cn_zip_4d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_4d_avg']
new_df['Amount_cn_zip_4d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_7d_sum')
new_df['Amount_cn_zip_7d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_7d_avg']
new_df['Amount_cn_zip_7d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_7d_avg']
new_df['Amount_cn_zip_7d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_14d_sum')
new_df['Amount_cn_zip_14d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_14d_avg']
new_df['Amount_cn_zip_14d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_14d_avg']
new_df['Amount_cn_zip_14d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch zip'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merch zip','Date'], rsuffix = '_cn_zip_30d_sum')
new_df['Amount_cn_zip_30d_act-avg'] = new_df['Amount_cn_zip_actual'] - new_df['Amount_cn_zip_30d_avg']
new_df['Amount_cn_zip_30d_act/avg'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_30d_avg']
new_df['Amount_cn_zip_30d_act/sum'] = new_df['Amount_cn_zip_actual']/new_df['Amount_cn_zip_30d_sum']

In [ ]:
df2 = df.groupby(['Cardnum','Merch state','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = new_df.join(df.groupby(['Cardnum','Merch state','Date'])['Amount'].sum(),on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_actual')

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '1D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_1d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '1D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_1d_sum')
new_df['Amount_cn_st_1d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_1d_avg']
new_df['Amount_cn_st_1d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_1d_avg']
new_df['Amount_cn_st_1d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_1d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '4D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_4d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '4D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_4d_sum')
new_df['Amount_cn_st_4d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_4d_avg']
new_df['Amount_cn_st_4d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_4d_avg']
new_df['Amount_cn_st_4d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_4d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '7D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_7d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '7D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_7d_sum')
new_df['Amount_cn_st_7d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_7d_avg']
new_df['Amount_cn_st_7d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_7d_avg']
new_df['Amount_cn_st_7d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_7d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '14D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_14d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '14D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_14d_sum')
new_df['Amount_cn_st_14d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_14d_avg']
new_df['Amount_cn_st_14d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_14d_avg']
new_df['Amount_cn_st_14d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_14d_sum']

new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '30D', closed = 'left').mean(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_30d_avg')
new_df = new_df.join(df2.groupby(['Cardnum','Merch state'])['Amount'].rolling(window = '30D', closed = 'left').sum(), on = ['Cardnum','Merch state','Date'], rsuffix = '_cn_st_30d_sum')
new_df['Amount_cn_st_30d_act-avg'] = new_df['Amount_cn_st_actual'] - new_df['Amount_cn_st_30d_avg']
new_df['Amount_cn_st_30d_act/avg'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_30d_avg']
new_df['Amount_cn_st_30d_act/sum'] = new_df['Amount_cn_st_actual']/new_df['Amount_cn_st_30d_sum']

In [ ]:
### frequency variables
new_df['Date'] = new_df['Date'].astype('datetime64[ns]')

# Cardnum
df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_1d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_4d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_7d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_14d'}), on = ['Cardnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_30d'}), on = ['Cardnum','Date'])

# Merchnum
df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_1d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_4d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_7d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_14d'}), on = ['Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_mn_30d'}), on = ['Merchnum','Date'])


In [ ]:
# Cardnum & Merchnum
df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_1d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_4d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_7d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_14d'}), on = ['Cardnum','Merchnum','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_mn_30d'}), on = ['Cardnum','Merchnum','Date'])

# Cardnum & Merch zip
df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_1d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_4d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_7d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_14d'}), on = ['Cardnum','Merch zip','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_zip_30d'}), on = ['Cardnum','Merch zip','Date'])


In [ ]:
# Cardnum & Merch state
df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_1d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '4D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_4d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '7D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_7d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '14D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_14d'}), on = ['Cardnum','Merch state','Date'])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '30D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'],keep = 'first')
new_df = new_df.merge(df3.rename(columns = {'Recnum':'Freq_cn_st_30d'}), on = ['Cardnum','Merch state','Date'])

In [ ]:
len(list(new_df))

165

In [ ]:
# Days since expert variables
df3 = df.set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Date'])
df3['Date_lag'] = df3.groupby('Cardnum')['Date'].shift(1)
df3['last_cn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Date','last_cn']])

df3 = df.set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Merchnum','Date'])
df3['Date_lag'] = df3.groupby('Merchnum')['Date'].shift(1)
df3['last_mn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Merchnum','Date','last_mn']])

df3 = df.set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merchnum','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merchnum'])['Date'].shift(1)
df3['last_cn_mn'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merchnum','Date','last_cn_mn']])

df3 = df.set_index('Date').groupby(['Cardnum','Merch zip'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch zip','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merch zip'])['Date'].shift(1)
df3['last_cn_zip'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merch zip','Date','last_cn_zip']])

df3 = df.set_index('Date').groupby(['Cardnum','Merch state'])['Recnum'].rolling(window = '1D', closed = 'left').count().to_frame().reset_index().drop_duplicates(subset = ['Cardnum','Merch state','Date'])
df3['Date_lag'] = df3.groupby(['Cardnum','Merch state'])['Date'].shift(1)
df3['last_cn_st'] = (df3['Date'] - df3['Date_lag']).dt.days
new_df = new_df.merge(df3[['Cardnum','Merch state','Date','last_cn_st']])


In [ ]:
# Velocity deviation
new_df['velo_cn_avg7d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_7d']/7)
new_df['velo_cn_avg14d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_14d']/14)
new_df['velo_cn_avg30d'] = new_df['Freq_cn_1d']/(new_df['Freq_cn_30d']/30)
new_df['velo_mn_avg7d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_7d']/7)
new_df['velo_mn_avg14d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_14d']/14)
new_df['velo_mn_avg30d'] = new_df['Freq_mn_1d']/(new_df['Freq_mn_30d']/30)

In [ ]:
# fill na with 0
new_df = new_df.fillna(0)

# export amount expert variables
new_df.to_csv('AmountVariables.csv')

In [3]:
new_df = pd.read_csv('AmountVariables.csv')

In [4]:
## add a dummy variable as for sanity checks
new_df['random'] = np.random.randint(0, 100, new_df.shape[0])

In [5]:
## calculate ks score for all variables
from scipy.stats import ks_2samp

ks_df = pd.DataFrame()
# ks_df['col'] = list(new_df)[10,:]
ks_df['col'] = list(new_df[new_df.columns[~new_df.columns.isin(['Amount_cn_actual','Amount_mn_actual','Amount_cn_mn_actual','Amount_cn_zip_actual','Amount_cn_st_actual'])]])[10:]

ks_df['p_value'] = ''
ks_df['ks_score'] = ''
for i in range(len(ks_df)):
    ks_df['ks_score'][i] = ks_2samp(new_df[new_df['Fraud'] == 0][ks_df['col'][i]], new_df[new_df['Fraud'] == 1][ks_df['col'][i]])[0]
    ks_df['p_value'][i] = ks_2samp(new_df[new_df['Fraud'] == 0][ks_df['col'][i]], new_df[new_df['Fraud'] == 1][ks_df['col'][i]])[1]


In [6]:
## calculate fraud detection rate for all variables
def fdr_cal(row):
    col_name = row['col']
    df_temp1 = new_df.sort_values(col_name, ascending = False)
    df_temp_top1 = df_temp1[:round(len(new_df)*0.03)]
    fdr1 = sum(df_temp_top1['Fraud'])/sum(df_temp1['Fraud'])
    df_temp2 = new_df.sort_values(col_name, ascending = True)
    df_temp_top2 = df_temp2[:round(len(new_df)*0.03)]
    fdr2 = sum(df_temp_top2['Fraud'])/sum(df_temp2['Fraud'])
    return max(fdr1, fdr2)

In [7]:
ks_df['fdr'] = ks_df.apply(lambda row: fdr_cal(row), axis=1)

In [8]:
## sort variables by fdr
fdr_sorted_list = ks_df.sort_values('fdr', ascending = False)
# fdr_sorted_list.to_pickle('fdr_sorted_list.pkl')

In [9]:
## sort variables by ks_score
ks_score_sorted_list = ks_df.sort_values('ks_score', ascending = False)
# ks_score_sorted_list.to_pickle('ks_score_sorted_list.pkl')

In [10]:
# ks_df['ks_rank'] = ks_df['ks_score'].rank(ascending = False)
# ks_df['fdr_rank'] = ks_df['fdr'].rank(ascending = False)
# ks_df['avg_rank'] = ks_df['ks_rank'] * 0.6 + ks_df['fdr_rank'] * 0.4 

In [11]:
## Divide into OOT and Train/Test

## Standardizes in-place
def standardize(df, colnames):
    for col in colnames:
        tmp_sd=df[col].std()
        tmp_mean=df[col].mean()
        df[col] = (df[col]-tmp_mean) / tmp_sd
standardize(new_df,list(new_df)[11:])

int_df = new_df[new_df["Date"] < new_df["Date"][84461]] 
oot_df = new_df[new_df["Date"] > new_df["Date"][84288]]

In [12]:
## calculate ks score for all variables
def fdr_cal(row, df):
    col_name = row['col']
    df_temp = df.sort_values(col_name, ascending = False)
    df_temp_top = df_temp[:round(len(df)*0.03)]
    df_temp_bot = df_temp.tail(round(len(df)*0.03))
    top_fdr= sum(df_temp_top['Fraud'])/sum(df_temp['Fraud'])
    bot_fdr= sum(df_temp_bot['Fraud'])/sum(df_temp['Fraud'])
#     print(str(top_fdr)+ " "+str(bot_fdr))
    return(max(top_fdr, bot_fdr))

def ks_fdr(df):
    ks_df = pd.DataFrame()
    ks_df['col'] = list(df)[10:]
    ks_df['p_value'] = ''
    ks_df['ks_score'] = ''
    # Calc KS
    for i in range(len(ks_df)):
        ks_df['ks_score'][i] = ks_2samp(df[df['Fraud'] == 0][ks_df['col'][i]], df[df['Fraud'] == 1][ks_df['col'][i]])[0]
        ks_df['p_value'][i] = ks_2samp(df[df['Fraud'] == 0][ks_df['col'][i]], df[df['Fraud'] == 1][ks_df['col'][i]])[1]
    # Calc FDR
    ks_df['fdr'] = ks_df.apply(lambda row: fdr_cal(row, df), axis=1)
    return ks_df

In [13]:
ks_fdr_int = ks_fdr(int_df)
ks_fdr_oot = ks_fdr(oot_df)

In [14]:
ks_fdr_int['ks_rank'] = ks_fdr_int['ks_score'].rank(ascending = False)
ks_fdr_int['fdr_rank'] = ks_fdr_int['fdr'].rank(ascending = False)
ks_fdr_int['avg_rank'] = ks_fdr_int['ks_rank'] * 0.6 + ks_fdr_int['fdr_rank'] * 0.4

In [15]:
top_features = ks_fdr_int.sort_values("avg_rank")["col"].head(80)
top_features

0                          Fraud
79          Amount_cn_zip_actual
53           Amount_cn_mn_actual
105          Amount_cn_st_actual
27              Amount_mn_actual
1               Amount_cn_actual
24         Amount_cn_30d_act-avg
50         Amount_mn_30d_act-avg
19         Amount_cn_14d_act-avg
45         Amount_mn_14d_act-avg
40          Amount_mn_7d_act-avg
14          Amount_cn_7d_act-avg
117          Amount_cn_st_7d_sum
35          Amount_mn_4d_act-avg
112          Amount_cn_st_4d_sum
111          Amount_cn_st_4d_avg
86          Amount_cn_zip_4d_sum
18             Amount_cn_14d_sum
85          Amount_cn_zip_4d_avg
13              Amount_cn_7d_sum
122         Amount_cn_st_14d_sum
59           Amount_cn_mn_4d_avg
128     Amount_cn_st_30d_act-avg
60           Amount_cn_mn_4d_sum
116          Amount_cn_st_7d_avg
91          Amount_cn_zip_7d_sum
8               Amount_cn_4d_sum
7               Amount_cn_4d_avg
51         Amount_mn_30d_act/avg
127         Amount_cn_st_30d_sum
          

In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(solver="liblinear", max_iter=300)
selector = RFE(estimator, 20, step=1)
selector = selector.fit(int_df[top_features], int_df["Fraud"])
final_features=top_features[selector.support_]
selected_df = pd.DataFrame(int_df, columns=list(final_features))

In [17]:
selected_df.to_csv("selected_df.csv")

In [18]:
selected_df=pd.read_csv("selected_df.csv")

train, test = train_test_split(selected_df, test_size=0.3)

In [19]:
sum(oot_df["Fraud"])

175

In [61]:
##### --------- Rogers Models RF and SVM ------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

features=['Amount_cn_st_actual', 'Amount_mn_30d_act-avg', 'Amount_cn_14d_act-avg',
 'Amount_mn_7d_act-avg', 'Amount_cn_7d_act-avg', 'Amount_cn_st_7d_sum',
 'Amount_cn_st_4d_avg', 'Amount_cn_14d_sum', 'Amount_cn_st_30d_act-avg',
 'Amount_cn_st_7d_avg', 'Amount_mn_30d_act/avg', 'Amount_cn_30d_act/avg',
 'Amount_cn_mn_7d_sum', 'Amount_cn_30d_sum', 'Amount_cn_30d_act/sum',
 'Amount_cn_zip_30d_avg', 'Amount_cn_zip_30d_sum', 'Amount_cn_st_7d_act-avg',
 'Amount_mn_1d_avg']

rf = RandomForestClassifier()
## RF Hyperparams
max_depth = [5,10,20,30]
max_depth.append(None)
rf_grid = {'n_estimators': [100, 500,1500],
           'max_features': ['auto', 'sqrt'],
           'criterion':['gini','entropy'],
           'max_depth': max_depth,
           'min_samples_split': [2, 5, 10],
           'min_samples_leaf':[2, 5, 10, 15],
           'bootstrap': [True]}
rf_best = {'n_estimators': [100],
         'min_samples_split': [2],
         'min_samples_leaf': [4],
         'max_features': ['auto'],
         'max_depth': [90],
         'bootstrap': [True]}
# print(rf_grid)
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = rf_grid, 
                               n_iter = 100, 
                               cv = 2,
                               verbose=1, 
                               random_state=123, 
                               n_jobs = -1)
rf_random.fit(train[list(train)[2:]], train["Fraud"])
rf_random.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 80.0min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 83.6min finished


{'n_estimators': 1500,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 20,
 'criterion': 'entropy',
 'bootstrap': True}

In [114]:
import warnings
warnings.filterwarnings('ignore')
def metrics_cal(mod):
    train["preds"]=""
    test["preds"]=""
    oot_df["preds"]=""
    
    train["preds"]=mod.predict(train[features])
    test["preds"]=mod.predict(test[features])
    oot_df["preds"]=mod.predict(oot_df[features])
    
    print("accuracy")
    ra_=metrics.accuracy_score(train["Fraud"], train["preds"])
    ta_=metrics.accuracy_score(test["Fraud"], test["preds"])
    oa_=metrics.accuracy_score(oot_df["Fraud"], oot_df["preds"])
    print(str(ra_) + str(ta_) + str(oa_))
    
    print("auc")
    rc_=metrics.roc_auc_score(train["Fraud"], train["preds"])
    tc_=metrics.roc_auc_score(test["Fraud"], test["preds"])
    oc_=metrics.roc_auc_score(oot_df["Fraud"], oot_df["preds"])
    print(str(rc_) + str(tc_) + str(oc_))
    
    train["preds"]=mod.predict_proba(train[features])[:,1]
    test["preds"]=mod.predict_proba(test[features])[:,1]
    oot_df["preds"]=mod.predict_proba(oot_df[features])[:,1]
    
    print("fdr")
    
    for df in [train,test,oot_df]:
        df_temp = df.sort_values("preds", ascending = False)
        df_temp_top = df_temp[:round(len(df)*0.03)]
        df_temp_bot = df_temp.tail(round(len(df)*0.03))
        top_fdr= sum(df_temp_top['Fraud'])/sum(df_temp['Fraud'])
        bot_fdr= sum(df_temp_bot['Fraud'])/sum(df_temp['Fraud'])
        print(max(top_fdr, bot_fdr))

# rf_preds=rf.predict_proba(test[list(train)[2:]])[:,1]

metrics_cal(rf_random)

accuracy
0.99878201441282940.99672389974343790.9855627578078963
auc
0.94748447586128440.84770647926973440.5940358891633
fdr
1.0
0.8932806324110671
0.6514285714285715


In [91]:
from sklearn.svm import SVC  
svc = SVC(kernel="rbf",probability=True)    
svm_grid = {'kernel': ["linear", "rbf"],
            'degree': [2, 3, 4],
            'gamma': ["auto"],
            'shrinking': [True,False],
            'class_weight': ["balanced"]}

svm_best= {'shrinking': [True],
           'kernel': ['rbf'],
           'gamma': ['auto'],
           'degree': [2],
           'class_weight': ["balanced"]}

svm_random = RandomizedSearchCV(estimator = svc, 
                                param_distributions = svm_best,
                                n_jobs=-1,
                                n_iter = 50, 
                                cv = 3, 
                                verbose=1, 
                                random_state=123)

svm_random.fit(train[features], train["Fraud"])
# svc.fit(train[features], train["Fraud"])

# svm_random.best_params_
metrics_cal(svm_random)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  6.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=50, n_jobs=-1,
          param_distributions={'shrinking': [True], 'kernel': ['rbf'], 'gamma': ['auto'], 'degree': [2], 'class_weight': ['balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [122]:
trainpred = pd.read_csv('train_pred.csv')
testpred = pd.read_csv('test_pred.csv')
ootpred = pd.read_csv('oot_pred.csv')

trainpred["Fraud"] = train["Fraud"].values
testpred["Fraud"] = test["Fraud"].values
ootpred["Fraud"] = oot_df["Fraud"].values

trainpred["rf"] = rf_random.predict_proba(train[features])[:,1]
testpred["rf"] = rf_random.predict_proba(test[features])[:,1]
ootpred["rf"] = rf_random.predict_proba(oot_df[features])[:,1]

trainpred["svm"] = svm_random.predict_proba(train[features])[:,1]
testpred["svm"] = svm_random.predict_proba(test[features])[:,1]
ootpred["svm"] = svm_random.predict_proba(oot_df[features])[:,1]

trainpred.to_csv('train_pred.csv')
testpred.to_csv('test_pred.csv')
ootpred.to_csv('oot_pred.csv')

In [121]:
ootpred

Unnamed: 0     logit            nn  Fraud            rf       svm
0               0  0.107786  3.595859e-05      0  0.000000e+00  0.000647
1               1  0.130642  2.369101e-12      0  8.571429e-04  0.000563
2               2  0.127210  9.193251e-10      0  0.000000e+00  0.002066
3               3  0.110270  2.767875e-10      0  0.000000e+00  0.001917
4               4  0.110270  2.767875e-10      0  0.000000e+00  0.001917
5               5  0.136699  7.026951e-09      0  0.000000e+00  0.002663
6               6  0.143670  1.910958e-08      0  0.000000e+00  0.002186
7               7  0.103450  9.031143e-09      0  0.000000e+00  0.001346
8               8  0.157458  4.985338e-10      0  0.000000e+00  0.001848
9               9  0.088376  3.262635e-06      0  7.222222e-04  0.001018
10             10  0.120201  1.093370e-01      0  0.000000e+00  0.000717
11             11  0.082974  5.262129e-04      0  0.000000e+00  0.000366
12             12  0.003713  4.317163e-24      0  8.400000e-03  0.000576
13             13  0.003903  6.785476e-15      0  1.909506e-02  0.019356
14             14  0.003973  2.130311e-11      0  2.027778e-02  0.001296
15             15  0.386017  2.053241e-07      0  1.830210e-05  0.011735
16             16  0.403816  4.862706e-10      0  1.845746e-03  0.002719
17             17  0.387188  1.416947e-06      0  4.444444e-04  0.000881
18             18  0.256521  2.296550e-08      0  1.134369e-03  0.002347
19             19  0.248023  2.836561e-06      0  4.471897e-04  0.002791
20             20  0.244198  7.517147e-07      0  1.285609e-03  0.000902
21             21  0.369061  1.588287e-05      0  1.111111e-03  0.031454
22             22  0.022401  3.109761e-05      0  8.849048e-02  0.035282
23             23  0.651074  1.044499e-15      0  2.179692e-01  0.000045
24             24  0.013979  5.638237e-27      0  2.633024e-02  0.000317
25             25  0.165209  1.335783e-03      0  2.114105e-06  0.000880
26             26  0.246752  8.410903e-10      0  2.667143e-02  0.057577
27             27  0.184902  4.210006e-02      0  5.093952e-03  0.000887
28             28  0.142608  2.767180e-03      0  3.351881e-06  0.000702
29             29  0.136602  5.423072e-06      0  0.000000e+00  0.000191
...           ...       ...           ...    ...           ...       ...
10152       10152  0.102555  5.351600e-03      0  0.000000e+00  0.000853
10153       10153  0.266878  7.296583e-05      0  1.148105e-02  0.005073
10154       10154  0.051426  8.353337e-07      0  6.666667e-04  0.000201
10155       10155  0.112007  2.071773e-01      0  9.470805e-07  0.001120
10156       10156  0.221950  2.348427e-10      0  2.607741e-03  0.000122
10157       10157  0.367535  1.055938e-16      0  3.035793e-03  0.000073
10158       10158  0.367535  1.055938e-16      0  3.035793e-03  0.000073
10159       10159  0.367535  1.055938e-16      0  3.035793e-03  0.000073
10160       10160  0.209962  3.386096e-14      0  0.000000e+00  0.000043
10161       10161  0.260925  7.160875e-10      0  1.543660e-05  0.000042
10162       10162  0.115253  1.546451e-01      0  2.454590e-07  0.001020
10163       10163  0.115253  1.546451e-01      0  2.454590e-07  0.001020
10164       10164  0.105884  9.945665e-04      0  0.000000e+00  0.000590
10165       10165  0.127496  3.355024e-10      0  3.346316e-04  0.000555
10166       10166  0.127425  3.237280e-08      0  4.045307e-07  0.002392
10167       10167  0.073933  1.884316e-06      0  0.000000e+00  0.000135
10168       10168  0.100934  2.160274e-03      0  0.000000e+00  0.000934
10169       10169  0.104483  2.476063e-07      0  1.211111e-03  0.000284
10170       10170  0.062430  5.761844e-05      0  0.000000e+00  0.000160
10171       10171  0.107572  1.699326e-02      0  3.333333e-04  0.001056
10172       10172  0.084825  1.361781e-05      0  2.454590e-07  0.000485
10173       10173  0.109205  1.817594e-01      0  8.042570e-05  0.001050
10174       10174  0.242956  1.391151e-05      0  1.155556e-03  0

In [115]:
svm_random.predict_proba(oot_df[features])[:,1]

array([0.00064694, 0.00056257, 0.00206625, ..., 0.00101721, 0.00534818,
       0.0002423 ])